<a href="https://colab.research.google.com/github/secutron/FixMatch-pytorch/blob/master/ds_ViT_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 15SEP21 HKim
# Exception: Installed CUDA version 11.0 does not match the version torch was compiled with 10.2, unable to compile cuda/cpp extensions without a matching cuda version.
!pip install torch==1.7.1+cu110  -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.7 MB/s eta 0:03:13tcmalloc: large alloc 1147494400 bytes == 0x55f9cd1dc000 @  0x7fd20155f615 0x55f993ceb02c 0x55f993dcb17a 0x55f993cede4d 0x55f993ddfc0d 0x55f993d620d8 0x55f993d5cc35 0x55f993cef73a 0x55f993d61f40 0x55f993d5cc35 0x55f993cef73a 0x55f993d5e93b 0x55f993de0a56 0x55f993d5dfb3 0x55f993de0a56 0x55f993d5dfb3 0x55f993de0a56 0x55f993d5dfb3 0x55f993cefb99 0x55f993d32e79 0x55f993cee7b2 0x55f993d61e65 0x55f993d5cc35 0x55f993cef73a 0x55f993d5e93b 0x55f993d5cc35 0x55f993cef73a 0x55f993d5db0e 0x55f993cef65a 0x55f993d5dd67 0x55f993d5cc35
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:22tcmalloc: large alloc 1434370048 bytes == 0x55fa11832000 @  0x7fd20155f615 0x55f993ceb02c 0x55f993dcb17a 0x55f993cede4d 0x55f993ddfc0d 0x55f993d620d8 0x55f993d5cc35 0x55f993cef73a 0x55f993d61f40 0x55f993d5cc35 0x55f993cef73a 0x55f993d5e93b 0x55f993de0a56 0x55f993d5df

In [3]:
# !pip install fairscale
!pip install deepspeed
!pip install einops

In [4]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
cpu_adam ............... [NO] ....... [OKAY]
fused_adam ............. [NO] ....... [OKAY]
fused_lamb ............. [NO] ....... [OKAY]
sparse_attn ............ [NO] ....... [OKAY]
transformer ............ [NO] ....... [OKAY]
stochastic_transformer . [NO] ....... [OKAY]
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the 

In [5]:
%%writefile ds_config.json

 {
   "train_batch_size": 4,
   "steps_per_print": 2000,
   "optimizer": {
     "type": "Adam",
     "params": {
       "lr": 0.001,
       "betas": [
         0.8,
         0.999
       ],
       "eps": 1e-8,
       "weight_decay": 3e-7
     }
   },
   "scheduler": {
     "type": "WarmupLR",
     "params": {
       "warmup_min_lr": 0,
       "warmup_max_lr": 0.001,
       "warmup_num_steps": 1000
     }
   },
   "wall_clock_breakdown": false
 }



Writing ds_config.json


In [6]:
%%writefile train_cifar10.py

import argparse
import deepspeed
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import CIFAR10
from time import perf_counter

def add_argument():
    """
    https://www.deepspeed.ai/tutorials/cifar-10/
    """
    parser=argparse.ArgumentParser(description='CIFAR')

    # data
    # cuda
    parser.add_argument('--with_cuda', default=False, action='store_true',
                        help='use CPU in case there\'s no GPU support')
    parser.add_argument('--use_ema', default=False, action='store_true',
                        help='whether use exponential moving average')

    # train
    parser.add_argument('-b', '--batch_size', default=512, type=int,
                        help='mini-batch size (default: 32)')
    parser.add_argument('-e', '--epochs', default=30, type=int,
                        help='number of total epochs (default: 30)')
    parser.add_argument('--local_rank', type=int, default=-1,
                    help='local rank passed from distributed launcher')

    # Include DeepSpeed configuration arguments
    parser = deepspeed.add_config_arguments(parser)

    return parser.parse_args()

"""
https://yhkim4504.tistory.com/5
https://github.com/lucidrains/vit-pytorch/blob/main/vit_pytorch/vit_pytorch.py
"""

import torch
import torch.nn.functional as F
from einops import rearrange, repeat
from torch import nn

MIN_NUM_PATCHES = 16

class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask = None):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), qkv)

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale
        mask_value = -torch.finfo(dots.dtype).max

        if mask is not None:
            mask = F.pad(mask.flatten(1), (1, 0), value = True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, mask_value)
            del mask

        attn = dots.softmax(dim=-1)

        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out =  self.to_out(out)
        return out

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Residual(PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout)))
            ]))
    def forward(self, x, mask = None):
        for attn, ff in self.layers:
            x = attn(x, mask = mask)
            x = ff(x)
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        assert num_patches > MIN_NUM_PATCHES, f'your number of patches ({num_patches}) is way too small for attention to be effective (at least 16). Try decreasing your patch size'
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img, mask = None):
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = p)
        x = self.patch_to_embedding(x)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x, mask)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

def main():
    args = add_argument()
    dataset = CIFAR10('.', download=True, transform=ToTensor())
    trainloader = torch.utils.data.DataLoader(dataset,
                                batch_size=args.batch_size,
                                shuffle=True,
                                num_workers=8)
    huge_model = ViT(
        image_size=32,
        patch_size=4,
        num_classes=10,
        dim=512,
        depth=8,
        heads=8,
        mlp_dim=2048,
        dropout=0.1,
        emb_dropout=0.1
    )
    lr = 0.001
    warmup_steps = 1000
    remain_steps = (args.epochs * len(trainloader) - warmup_steps)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        huge_model.parameters(),
        lr=lr,
        betas=(0.8, 0.999),
        eps=1e-8,
        weight_decay=3e-7)
    torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda epoch: (epoch + 1) / warmup_steps * lr if epoch < warmup_steps else (epoch - warmup_steps) * lr / remain_steps)
    model_engine, _, trainloader_ds, _ = deepspeed.initialize(
        args=args,
        model=huge_model,
        model_parameters=huge_model.parameters(),
        training_data=dataset)

    # training w/ DeepSpeed
    start_time = perf_counter()
    for data in trainloader_ds:
            inputs = data[0].to(model_engine.device)
            labels = data[1].to(model_engine.device)

            outputs = model_engine(inputs)
            loss = criterion(outputs, labels)

            model_engine.backward(loss)
            model_engine.step()
    ds_time = (perf_counter() - start_time) / 60
    print('###################################################################')
    print(f'Training CIFAR10 using DeepSpeed used {ds_time:.3f} minutes')

    # regular training
    model = huge_model.to(device)
    start_time = perf_counter()
    for data in trainloader:
        inputs = data[0].to(device)
        labels = data[1].to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    no_ds_time = (perf_counter() - start_time) / 60
    print('###################################################################')
    print(f'Training CIFAR10 without using DeepSpeed used {no_ds_time:.3f} minutes')
    print('###################################################################')
    print(f'DeepSpeed accelerated training by {no_ds_time - ds_time:.3f} minutes')        

if __name__ == '__main__':
    main()    

Writing train_cifar10.py


In [7]:
!deepspeed train_cifar10.py --deepspeed_config ds_config.json

[2021-09-15 08:34:17,171] [WARNING] [runner.py:122:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2021-09-15 08:34:17,196] [INFO] [runner.py:360:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 train_cifar10.py --deepspeed_config ds_config.json
[2021-09-15 08:34:18,096] [INFO] [launch.py:73:main] 0 NCCL_VERSION 2.7.8
[2021-09-15 08:34:18,096] [INFO] [launch.py:80:main] WORLD INFO DICT: {'localhost': [0]}
[2021-09-15 08:34:18,097] [INFO] [launch.py:89:main] nnodes=1, num_local_procs=1, node_rank=0
[2021-09-15 08:34:18,097] [INFO] [launch.py:101:main] global_rank_mapping=defaultdict(<class 'list'>, {'localhost': [0]})
[2021-09-15 08:34:18,097] [INFO] [launch.py:102:main] dist_world_size=1
[2021-09-15 08:34:18,097] [INFO] [launch.py:105:main] Setting CUDA_VISIBLE_DEVICES=0
170499072it [00:02, 62596756.56it/s]                   
Extracting ./cifar-1

In [1]:
import os

gpu_gtg = False
if int(os.environ.get("COLAB_GPU")) > 0:
    gpu_gtg = "COLAB_GPU" in os.environ

tpu_gtg = "COLAB_TPU_ADDR" in os.environ

if tpu_gtg: # tpu
    print("TPU")
    #VERSION = "nightly"

    # https://github.com/pytorch/builder/pull/750
    VERSION = "20210304" # was 20200607" 

    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

In [2]:
!pip install --pre pytorch-ignite

     |████████████████████████████████| 233 kB 5.3 MB/s 


In [5]:
# !pip install fairscale
!pip install deepspeed

     |████████████████████████████████| 477 kB 5.3 MB/s 
     |████████████████████████████████| 216 kB 28.7 MB/s 
  Using cached ninja-1.10.2-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (108 kB)
     |████████████████████████████████| 15.2 MB 33.1 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.2-py3-none-any.whl size=479879 sha256=21dc2fa500eb5e1c62c019e733cc66f94db9aa0cc157e96084c29345f0c04161
  Stored in directory: /root/.cache/pip/wheels/fe/57/72/d7669268042846842d93c36e1447bc7cd8603f77bbbd8ee62b
Successfully built deepspeed


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import torchsummary

import ignite
import ignite.distributed as idist
from ignite.engine import Engine, Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.utils import setup_logger

In [ ]:
def training(local_rank, config, **kwargs):
    print("local rank: ", local_rank)

    ###########################################################
    # 데이터 준비
    train_transform = transforms.Compose(
        [
            transforms.Pad(4),
            transforms.RandomCrop(32, fill=128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

    test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])

    if idist.get_local_rank() > 0:
        idist.barrier()

    trainset = torchvision.datasets.CIFAR10(root=config["data_path"], train=True, download=True, transform=train_transform)
    testset = torchvision.datasets.CIFAR10(root=config["data_path"], train=False, download=True, transform=test_transform)

    if idist.get_local_rank() == 0:
        idist.barrier()

    trainloader = idist.auto_dataloader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"], drop_last=True)
    testloader = idist.auto_dataloader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=config["num_workers"],)


    ###########################################################
    # 모델, 옵티마이저, 로스, 트레이너, 이밸류에이터
    num_classes = 10
    model = models.resnet18(num_classes = num_classes)
       
    model = idist.auto_model(model)
    optimizer = idist.auto_optim(optim.Adam(model.parameters(), lr=0.001))

    criterion = nn.CrossEntropyLoss().to(idist.device())

    trainer = create_supervised_trainer(model, optimizer, criterion, device=idist.device())
    trainer.logger = setup_logger("hkim-trainer")

    metrics = {
        'accuracy':Accuracy(),
        'ce':Loss(criterion),
    }

    val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=idist.device())
    val_evaluator.logger = setup_logger("hkim-val_evaluator")

    # track a running average of the scalar loss output for each batch.
    RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

    ###########################################################
    # 이벤트

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        state = val_evaluator.run(testloader)
        metrics = val_evaluator.state.metrics
        accuracy = metrics['accuracy']*100
        loss = metrics['ce']
        log_metrics(val_evaluator.logger, state.epoch, state.times["COMPLETED"], "validation evaluator", state.metrics)

    trainer.run(trainloader, max_epochs=config["num_epochs"])    

In [ ]:
config = {
    "seed": 543,
    "data_path" : "./cifar10",
    "output_path" : "./output-cifar10/",
    "model" : "resnet18",
    "batch_size" : 512,
    "momentum" : 0.9,
    "weight_decay" : 1e-4,
    "num_workers" : 2,
    "num_epochs" : 24,
    "learning_rate" : 0.4,
    "num_warmup_epochs" : 4,
    "validate_every" : 3, 
    "checkpoint_every" : 1000,
    "backend" : None, 
    "resume_from" : None, 
    "log_every_iters" : 15,
    "nproc_per_node" : None, 
    "stop_iteration" : None, 
    "with_amp" : False,
    "log_interval" : 10,
    "verbose_set" : False,
    "verbose_set2" : False,
    "verbose_loader" : False

}

if not (tpu_gtg or gpu_gtg): # cpu
    config["backend"] = 'gloo'
    config["nproc_per_node"] = 8
elif gpu_gtg: # gpu
    config["backend"] = 'nccl'
    config["nproc_per_node"] = 1
elif tpu_gtg: # tpu
    config["backend"] = 'xla-tpu'
    config["nproc_per_node"] = 8
else: # error
    raise RuntimeError("Unknown environment: tpu_gtg {}, gpu_gtg {}".format(tpu_gtg, gpu_gtg))

if config["backend"] == "xla-tpu" and config["with_amp"]:
    raise RuntimeError("The value of with_amp should be False if backend is xla")


dist_configs = {'nproc_per_node': config["nproc_per_node"], "start_method": "fork"}  

def log_metrics(logger, epoch, elapsed, tag, metrics):
    metrics_output = "\n".join([f"\t{k}: {v}" for k, v in metrics.items()])
    logger.info(f"\nEpoch {epoch} - Evaluation time (seconds): {elapsed:.2f} - {tag} metrics:\n {metrics_output}")

with idist.Parallel(backend=config["backend"], **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=1)

2021-09-13 07:43:17,782 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-09-13 07:43:17,784 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-13 07:43:17,786 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fa2a1e58cb0>' in 8 processes


local rank:  7
local rank:  2
local rank:  5
local rank:  1
local rank:  3
local rank:  6
local rank:  4
local rank:  0


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


2021-09-13 07:44:48,697 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 64, 'num_workers': 2, 'drop_last': True, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fa2b316a450>, 'pin_memory': False}
2021-09-13 07:44:48,719 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA
2021-09-13 07:44:48,740 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 64, 'num_workers': 2, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fa29afa2050>, 'pin_memory': False}
2021-09-13 07:44:48,753 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


2021-09-13 07:44:50,868 hkim-trainer INFO: Engine run starting with max_epochs=24.


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


2021-09-13 07:45:45,149 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-13 07:45:50,900 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:06
2021-09-13 07:45:50,918 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:06
2021-09-13 07:45:50,926 hkim-val_evaluator INFO: 
Epoch 1 - Evaluation time (seconds): 5.76 - validation evaluator metrics:
 	accuracy: 0.5083
	ce: 1.3657859375
2021-09-13 07:45:50,930 hkim-trainer INFO: Epoch[1] Complete. Time taken: 00:00:60
2021-09-13 07:46:23,184 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-13 07:46:26,740 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:03
2021-09-13 07:46:26,750 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:04
2021-09-13 07:46:26,759 hkim-val_evaluator INFO: 
Epoch 1 - Evaluation time (seconds): 3.54 - validation evaluator metrics:
 	accuracy: 0.5722
	ce: 1.19976494140625
2021-09-13 07:46:26,771 hkim-trainer INFO: Epoch[2] Com

## License


---


Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  


---